# Open Closed Principle

Las clases deben estar abiertas para extensión pero cerradas para modificación.


Tienes una aplicación de comercio electrónico con una clase `Pedido` que calcula los costos de envío, y todos los métodos de envío están incrustados dentro de la clase. Si necesitas añadir un nuevo método de envío, tienes que cambiar el código de la clase `Pedido`, arriesgándote a descomponerlo.


**Antes:** Tienes que cambiar la clase `Pedido` para añadir un nuevo método de envío.


In [1]:
class Order:
    def __init__(self, shipping: str, items: list) -> None:
        self.shipping: str = shipping
        self.items: list = items

    def get_total(self) -> float:
        total = sum([item["price"] for item in self.items])
        return total

    def get_total_weight(self) -> float:
        total_weight = sum([item["weight"] for item in self.items])
        return total_weight

    def get_shipping_cost(self) -> float:
        if self.shipping == "ground":
            if self.get_total() > 100:
                return 0
            return 1.5 * self.get_total_weight()
        elif self.shipping == "air":
            return 3 * self.get_total_weight()
        return 0

In [2]:
data_order: dict = [{"price": 10, "weight": 2}, {"price": 20, "weight": 3}]
order: Order = Order(shipping="air", items=data_order)
print("Total: ", order.get_total())
print("Total weight: ", order.get_total_weight())
print("Shipping cost: ", order.get_shipping_cost())

Total:  30
Total weight:  5
Shipping cost:  15


In [3]:
data_order_2: dict = [{"price": 90, "weight": 2}, {"price": 20, "weight": 3}]
order_2: Order = Order(shipping="ground", items=data_order_2)
print("Total: ", order_2.get_total())
print("Total weight: ", order_2.get_total_weight())
print("Shipping cost: ", order_2.get_shipping_cost())

Total:  110
Total weight:  5
Shipping cost:  0


Puedes resolver el problema aplicando el patrón `Strategy` (El cual aprenderemos a detalle más adelante). Empieza extrayendo métodos de envío y colocándolos dentro de clases separadas con una interfaz común.


**Después:** Con los nuevos cambios, añadir un nuevo método de envío es tan simple como añadir una nueva clase. Adicionalmente, los cambios en una clase no afectarán a las demás.

In [4]:
from abc import ABC, abstractmethod


class Shipping(ABC):
    @abstractmethod
    def get_cost(self, order: Order) -> float:
        pass


class GroundShipping(Shipping):
    def get_cost(self, order: Order) -> float:
        if order.get_total() > 100:
            return 0
        return 1.5 * order.get_total_weight()


class AirShipping(Shipping):
    def get_cost(self, order: Order) -> float:
        return 3 * order.get_total_weight()


class Order:
    def __init__(self, shipping: Shipping, items: list) -> None:
        self.shipping: Shipping = shipping
        self.items: list = items

    def get_total(self) -> float:
        total = sum([item["price"] for item in self.items])
        return total

    def get_total_weight(self) -> float:
        total_weight = sum([item["weight"] for item in self.items])
        return total_weight

    def get_shipping_cost(self) -> float:
        return self.shipping.get_cost(self)

In [5]:
data_order: dict = [{"price": 10, "weight": 2}, {"price": 20, "weight": 3}]
order: Order = Order(shipping=AirShipping(), items=data_order)
print("Total: ", order.get_total())
print("Total weight: ", order.get_total_weight())
print("Shipping cost: ", order.get_shipping_cost())

Total:  30
Total weight:  5
Shipping cost:  15


In [6]:
data_order_2: dict = [{"price": 90, "weight": 2}, {"price": 20, "weight": 3}]
order_2: Order = Order(shipping=GroundShipping(), items=data_order_2)
print("Total: ", order_2.get_total())
print("Total weight: ", order_2.get_total_weight())
print("Shipping cost: ", order_2.get_shipping_cost())

Total:  110
Total weight:  5
Shipping cost:  0


Ahora, cuando necesites implementar un nuevo método de envío, puedes derivar una nueva clase de la interfaz `Envíos` sin tocar el código de la clase `Pedido`. El código cliente de la clase `Pedido` vinculará los pedidos con un objeto de envío de la nueva clase cuando el usuario seleccione estos métodos de envío en la UI.

Suponiendo que agregaramos un nuevo método de envío llamado `Envío por Drone`, el código se vería de la siguiente manera:

In [7]:
class DroneShipping(Shipping):
    def get_cost(self, order: Order) -> float:
        return 4 * order.get_total_weight()

In [8]:
data_order_2: dict = [{"price": 90, "weight": 2}, {"price": 20, "weight": 3}]
order_2: Order = Order(shipping=DroneShipping(), items=data_order_2)
print("Total: ", order_2.get_total())
print("Total weight: ", order_2.get_total_weight())
print("Shipping cost: ", order_2.get_shipping_cost())

Total:  110
Total weight:  5
Shipping cost:  20


De esta manera, la clase `Pedido` no necesita ser modificada para añadir un nuevo método de envío. Simplemente se crea una nueva clase que implementa la interfaz `Envíos` y se agrega al código cliente. Aplicando el principio de abierto/cerrado, el código es más fácil de mantener y extender.

- El principio de abierto/cerrado se ve aplicado acá ya que la clase `Pedido` está cerrada para modificaciones, pero abierta para extensiones a través de la implementación de nuevas clases de envío.